<a href="https://colab.research.google.com/github/SCCSMARTCODE/Deep-Learning-00/blob/main/LeNet/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Overview: Handwritten Digit Classification with MNIST**

This project aims to develop a robust handwritten digit classification system utilizing the `MNIST dataset`. The primary focus is to explore the effects of dropout and weight decay techniques on model performance to enhance learning efficacy. We will be employing the ``LeNet`` architecture as the foundational framework for this classification task.

## **Objectives**:

 - [ ] To investigate the application of dropout and weight decay methods in preventing overfitting and improving the generalization of the model.
To implement and evaluate the ``LeNet`` architecture for handwritten digit recognition.

 - [ ] To analyze the performance metrics of the model under different configurations of dropout and weight decay.

 - [ ] To leverage Weights & Biases ``(WandB)`` for visualizing and monitoring the training metrics, facilitating a better understanding of the model's performance throughout the training process.


This document outlines the methodology and planned approach for successfully executing this project, aiming to achieve accurate and reliable classification of handwritten digits.

`Dependency Importation`

In [ ]:
%pip install wandb

In [2]:
import torch
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Pad, Compose
from torch import nn as nn
from torchsummary import summary
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR
import wandb

`Data PipeLine`

In [ ]:
basic_data_transform = Compose(
    [
        ToTensor(),
        Pad(padding=2)
    ]
)


raw_training_dataset = MNIST(root='.', transform=basic_data_transform, download=True)
raw_testing_dataset = MNIST(root='.', train=False, transform=basic_data_transform, download=False)

train_ds_len = 55000
valid_ds_len = 5000


training_dataset, validation_dataset = random_split(raw_training_dataset, [train_ds_len, valid_ds_len])

### Defining DataLoader
train_dl = DataLoader(training_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
test_dl = DataLoader(raw_testing_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True, drop_last=True)
valid_dl = DataLoader(validation_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True, drop_last=True)

`LeNet Model Defination & Initialization `

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=32),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.MaxPool2d(kernel_size=4),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=128),
            nn.MaxPool2d(kernel_size=2)
        )

        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 64),
            nn.Dropout1d(p=.3),

            nn.Linear(64, 10)
            )

    def forward(self, input):
        out = self.conv_layers(input)
        out = self.fc_layer(out)
        return out


network = LeNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network = network.to(device)
summary(network, input_size=(1,32,32),  batch_size=128)
network

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [128, 32, 32, 32]             320
              ReLU-2          [128, 32, 32, 32]               0
       BatchNorm2d-3          [128, 32, 32, 32]              64
         MaxPool2d-4          [128, 32, 16, 16]               0
            Conv2d-5          [128, 64, 16, 16]          18,496
              ReLU-6          [128, 64, 16, 16]               0
       BatchNorm2d-7          [128, 64, 16, 16]             128
         MaxPool2d-8            [128, 64, 4, 4]               0
            Conv2d-9           [128, 128, 4, 4]          73,856
             ReLU-10           [128, 128, 4, 4]               0
      BatchNorm2d-11           [128, 128, 4, 4]             256
        MaxPool2d-12           [128, 128, 2, 2]               0
          Flatten-13                 [128, 512]               0
           Linear-14                  [

LeNet(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=512, out_features=64, bias=True)
    (2): Dropout1d(p=0.3, inplace=Fa

`Hyperpaarameter & B.P Function Definition `



In [7]:
EPOCHS=10
LR=1e-4
MAX_LR=1e-2
BATCH_SIZE=128

criterion = nn.CrossEntropyLoss()
optimizer = Adam(params=network.parameters(), lr=LR, weight_decay=.002)
lr_scheduler = OneCycleLR(optimizer=optimizer, max_lr=MAX_LR, steps_per_epoch=BATCH_SIZE, epochs=EPOCHS)

`Accuracy function definition `

In [8]:
@torch.no_grad()
def accuracy(model, data_loader, criterion):
    acc_count = 0
    total_preds = 0
    total_loss = 0.0

    model.eval()

    for data, target in data_loader:
        data, target = data.to(device), target.to(device)
        pred = model(data)
        formated_pred = torch.argmax(pred, dim=1)

        acc_count += (formated_pred == target).sum().item()
        total_preds += len(target)

        loss = criterion(pred, target)
        total_loss += loss.item()

    if total_preds == 0:
        return 0.0, None

    avg_loss = total_loss / len(data_loader)
    accuracy_percentage = (acc_count / total_preds) * 100

    return accuracy_percentage, avg_loss

In [10]:
wandb.init(project="LeNet", config={
    "epochs": EPOCHS,
    "learning_rate": optimizer.param_groups[0]['lr'],
})


def train(model, epochs, train_dl, val_dl, criterion, optimizer, lr_scheduler):
    model.train()

    for epoch in range(epochs):
        for data, target in test_dl:
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            pred = model(data)

            loss = criterion(pred, target)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()

            wandb.log({
                "train_loss": loss.item(),
                "learning_rate": optimizer.param_groups[0]['lr']
            })


        ### validation phase
        val_acc, val_loss = accuracy(model, val_dl, criterion)
        wandb.log({"val_loss": val_loss, "val_acc": val_acc})

        wandb.watch(model, criterion, log="all")

    wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
train(network, EPOCHS, train_dl, valid_dl, criterion, optimizer, lr_scheduler)

learning_rate,▁▁▁▁▁▂▃▃▄▄▅▅▆▆▆▇██████████████▇▇▇▇▆▆▆▅▅▅
train_loss,█▄▂▂▁▁▁▁▁▁▁▂▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▆▂▆▁▃▇████
val_loss,▃▅▂█▅▁▁▁▁▁
learning_rate,0.00589
train_loss,0.02285
val_acc,97.09535
val_loss,0.09437


`Network Inference`

In [12]:
def test(model, test_loader, criterion):
    accuracy_percentage, avg_loss = accuracy(model, test_loader, criterion)

    print(f"Test Results:\n"
          f"{'='*30}\n"
          f"Accuracy: {accuracy_percentage:.2f}%\n"
          f"Average Loss: {avg_loss:.4f}\n"
          f"{'='*30}")

In [14]:
test(network, test_dl, criterion)

Test Results:
Accuracy: 99.16%
Average Loss: 0.0253


In [15]:
torch.save(network.state_dict(), '/content/drive/MyDrive/Deep Learning/LeNet/parameter.pth')